This notebook contains a class for processing an FMCW RADAR data capture. 

In [4]:
import h5py
import numpy as np
import os
import imageio
import matplotlib.pyplot as plt

In [5]:
class FMCWRADARDataCapture:
    """
    Class for handling the capture, processing, and saving of FMCW RADAR data.

    This class is designed to load Frequency-Modulated Continuous-Wave (FMCW) RADAR data from a specified HDF5 file,
    process the data into a usable format, and save it as a NumPy file (either .npy or .npz format).

    Attributes:
        file_path (str): Path to the HDF5 file containing the RADAR data.
    """

    def __init__(self, file_path):
        """
        Initializes the FMCWRADARDataCapture class with the specified file path.

        Args:
            file_path (str): Path to the HDF5 file to be loaded and processed.
        """
        
        if not os.path.isfile(file_path):
            raise FileNotFoundError(f"The file '{file_path}' does not exist.")

        self.file_path = file_path
        self.output_path = self.file_path.replace("_Data", "_Data_NP")

    def load_and_save(self, output_path=None, format='npy'):
        if output_path is None:
            output_path = self.output_path 
            output_path = os.path.splitext(output_path)[0]
            
        # Ensure the directory of the output path exists
        output_dir = os.path.dirname(output_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Open the HDF5 file
        with h5py.File(self.file_path, 'r') as file:
            # Reading parameters from the HDF5 file
            N = file['/BrdCfg/N'][()]
            Np = file['/BrdCfg/Np'][()]
            fs = file['/BrdCfg/fs'][()]
            TimUp = file['/BrdCfg/TimUp'][()]
            FreqStrt = file['/BrdCfg/FreqStrt'][()]
            FreqStop = file['/BrdCfg/FreqStop'][()]

            # Calculating effective bandwidth
            B = (FreqStop - FreqStrt) / 284 * 256
            range_resolution = 3e8 / (2 * B)

            # Reading the If signal
            If = file['/If'][:]

        # Parameters (you might need to adjust these)
        numFrames = 1000
        numChirpsPerFrame = 128
        numSamplesPerChirp = 256
        numAntennas = 4

        # Processing the If signal
        dataCubes = self.rawDataToDataCube(If, numFrames, numChirpsPerFrame, numSamplesPerChirp, numAntennas)
        
        print(dataCubes.shape)

        # Save data in the specified format
        if format == 'npy':
            np.save(output_path, dataCubes)
        elif format == 'npz':
            np.savez(output_path, dataCubes)
        else:
            raise ValueError("Unsupported format. Use 'npy' or 'npz'.")
        
        return dataCubes

    @staticmethod
    def rawDataToDataCube(rawData, numFrames, numChirpsPerFrame, numSamplesPerChirp, numAntennas):
        # Reshape and rearrange the rawData
        matrixData = rawData.T.reshape(numChirpsPerFrame * numSamplesPerChirp, numFrames * numAntennas)
        dataCubes = np.zeros((numFrames, numChirpsPerFrame, numSamplesPerChirp, numAntennas))

        for frame in range(numFrames):
            for antenna in range(numAntennas):
                chirps = matrixData[:, frame * numAntennas + antenna]
                chirpsMatrix = chirps.reshape(numSamplesPerChirp, numChirpsPerFrame)
                dataCubes[frame, :, :, antenna] = chirpsMatrix.T

        return dataCubes
    
    def range_doppler_processing(self, dataCube):
        """
        Processes each frame in the dataCube for each channel to generate Range-Doppler Maps.

        Args:
            dataCube (np.ndarray): The raw data cubes to be processed.

        Returns:
            np.ndarray: Array of processed Range-Doppler Maps for each channel.
        """
        n_frames, _, _, n_channels = dataCube.shape
        rdm_all_channels = []

        for channel_idx in range(n_channels):
            rdm_list = []
            for frame_idx in range(n_frames):
                # Extract current data for the frame and channel
                current_data = dataCube[frame_idx, :, :, channel_idx]

                # Apply 2D FFT and shift
                rdm = np.fft.fft2(current_data)
                rdm = np.fft.fftshift(rdm, axes=1)  # Shift along the Doppler axis (second axis in Python)

                # Take the absolute value
                rdm = np.abs(rdm)

                # Append to the list for the current channel
                rdm_list.append(rdm)

            # Append the result for the current channel
            rdm_all_channels.append(rdm_list)

        return np.array(rdm_all_channels)
    
    def create_gif(self, data, gif_path, duration=0.1):
        """
        Creates a GIF from the provided data.

        Args:
            data (np.ndarray): 3D or 4D array containing the image data.
            gif_path (str): Path to save the GIF.
            duration (float): Duration of each frame in the GIF.
        """
        with imageio.get_writer(gif_path, mode='I', duration=duration) as writer:
            if data.ndim == 3:  # If data is 3D, treat it as a sequence of 2D frames
                for i in range(data.shape[0]):
                    # Convert the data to an image (you might need to scale/normalize)
                    frame = data[i, :, :]
                    plt.imshow(frame, cmap='gray')  # Use appropriate colormap
                    plt.axis('off')  # Turn off axis
                    plt.savefig('temp_frame.png', bbox_inches='tight', pad_inches=0)
                    plt.close()
                    writer.append_data(imageio.imread('temp_frame.png'))
            elif data.ndim == 4:  # If data is 4D, process each channel separately
                for channel in range(data.shape[0]):
                    for i in range(data.shape[1]):
                        frame = data[channel, i, :, :]
                        plt.imshow(frame, cmap='gray')  # Use appropriate colormap
                        plt.axis('off')  # Turn off axis
                        plt.savefig('temp_frame.png', bbox_inches='tight', pad_inches=0)
                        plt.close()
                        writer.append_data(imageio.imread('temp_frame.png'))

        # Remove temporary frame image file
        os.remove('temp_frame.png')


